In [10]:
# Import os and disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

In [11]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Flatten, BatchNormalization, Dropout

In [12]:
places_data = pd.read_csv('final-dataset/main_dataset.csv')
places_data

,id,name,latitude,longitude,address,url,status,phone,primary-type,types,rating,rating-count,price-level,review 1,review 2,review 3,review 4,review 5
0,ChIJYcGr7GSb0S0RckePBrCWikw,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,-8.103137,115.090514,"Jl. Surapati No.12 14, Kp. Baru, Kec. Buleleng...",https://maps.google.com/?cid=5515386376437057394,OPERATIONAL,+62 362 3301500,hotel,"hotel, lodging",4.1,2581.0,PRICE_LEVEL_UNKNOWN,"It has quite small room, and the hallway is qu...",Few days ago My wife and me stayed 2 night at ...,"Because its cheap, cant complaint too much. Th...","Stayed here for 1 night, the room is small wit...",Good place to short stay located next Historic...
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,Warung Bik Juk,-8.106189,115.089961,"Kampung Kajanan, Buleleng, Buleleng Regency, B...",https://maps.google.com/?cid=2294119279772554194,OPERATIONAL,+62 819-1566-2913,indonesian_restaurant,"indonesian_restaurant, restaurant, food",4.5,648.0,PRICE_LEVEL_MODERATE,"Surprisingly, a really good warung that’s hidd...","So in love with the food, very simple but tast...",A hidden treasure in the heart of Singaraja Ci...,Nice warung on the side street. They were out ...,Nostalgic place at singaraja.. good and friend...
2,ChIJYyHbhgia0S0RzdjNXLmcf54,Harbour Tourist Area of Buleleng,-8.103931,115.090031,"Jl. Pelabuhan Buleleng, Kp. Bugis, Kec. Bulele...",https://maps.google.com/?cid=11421019499974744269,OPERATIONAL,+62 362 21342,tourist_attraction,"tourist_attraction, restaurant, food",4.3,2800.0,PRICE_LEVEL_UNKNOWN,"Only had a fleeting visit here, came by coach,...",The Harbour Tourist Area in Buleleng offers a ...,Its ine of nice place you must visit when you ...,"A historical place for the Balinese people, es...",It's a good place to relax with you friends an...
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,Abuela Coffee,-8.104751,115.087919,"Jl. Erlangga No.1, Kp. Bugis, Kec. Buleleng, K...",https://maps.google.com/?cid=10868728212674968854,OPERATIONAL,+62 858-5878-5361,coffee_shop,"coffee_shop, cafe, food, store",4.9,164.0,PRICE_LEVEL_UNKNOWN,"One word, underrated! How come place like this...",Coffee shop with good food and coffee. Very ni...,"Small cafe, the coffee is good, the dimsum is ...",Love this place 🫶\r\nGreat coffee ☕️\r\nLumpia...,Best coffee in Singaraja. Cappuccino was 20k a...
4,ChIJ63FmGgaa0S0RWD5dfwhjGHQ,Ikan Bakar Tanjung Alam Dermaga,-8.103087,115.088426,"Kampung Bugis, Buleleng, Buleleng Regency, Bal...",https://maps.google.com/?cid=8365545195988401752,OPERATIONAL,+62 852-3771-1414,indonesian_restaurant,"indonesian_restaurant, restaurant, food",4.4,138.0,PRICE_LEVEL_MODERATE,"We came here for dinner, food was good, i like...",The food was good. The view is wonderful. It's...,Nice place for dinner with family. Good place ...,This was the highlight of Singaraja. A nice co...,Always enjoyed our meal here. But for chili or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,ChIJoW-FX3tb0i0RswZrEuxU3dg,Karang Kembar Estate - Villa 2,-8.842963,115.184412,"Jl. Karang Kembar No.2, Banjar Jabapura, Kutuh...",https://maps.google.com/?cid=15626739654943508147,OPERATIONAL,+62 361 769071,lodging,lodging,4.5,59.0,PRICE_LEVEL_UNKNOWN,I went to this place with family and relatives...,Grand dame of the past...\r\n\r\nFantastic Sea...,Location: you just need go to the right from P...,"The amazing estate, wonderful view, like a par...",A nice Villa with a view to the Pandawa beach....
5364,ChIJHcVJyNhd0i0Rtif8ZHzZBr0,Gunung Payung Beach,-8.840777,115.199836,"Pantai Gunung Payung, Kutuh, Kec. Kuta Sel., K...",https://maps.google.com/?cid=13620813251322062774,OPERATIONAL,+62 811-3834-388,tourist_attraction,tourist_attraction,4.8,745.0,PRICE_LEVEL_UNKNOWN,The stairs are manageable or you can pay about...,One of the best beaches in Uluwatu area. It’s ...,You have two options for getting to the beach....,Private beach with access through Gunung Payun...,Goes there at around 3PM and it looks so good ...
5365,ChIJQ_u4

In [13]:
id_encoder = LabelEncoder()
place_ids_encoded = id_encoder.fit_transform(places_data['id'])

types_encoder = LabelEncoder()
place_types_encoded = types_encoder.fit_transform(places_data['types'])

In [14]:
places_data['combined-review'] = places_data[['review 1', 'review 2', 'review 3', 'review 4', 'review 5']].apply(lambda x: ' '.join(x), axis=1)

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(places_data['combined-review'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(places_data['combined-review'])
max_review_length = max(len(seq) for seq in sequences)

In [16]:
input_place_ids = Input(shape=(1,), name='place_id')
dense_place_ids = Dense(len(id_encoder.classes_), activation='relu')(input_place_ids)

input_place_types = Input(shape=(1,), name='place_type')
dense_place_types = Dense(len(types_encoder.classes_), activation='relu')(input_place_types)

input_place_reviews = Input(shape=(max_review_length,), name='place_review')
embedding_place_reviews = Embedding(input_dim=vocab_size, output_dim=32)(input_place_reviews)
lstm_place_reviews = LSTM(32)(embedding_place_reviews)

combined = Concatenate()([dense_place_ids, dense_place_types, lstm_place_reviews])

dense1 = Dense(128, activation='relu')(combined)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(32, activation='relu')(dense2)
output = Dense(1, activation='sigmoid')(dense3)

In [17]:
model = Model(inputs=[input_place_ids, input_place_types, input_place_reviews], outputs=output)
model.summary()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 place_review (InputLayer)   [(None, 1442)]               0         []                            
                                                                                                  
 place_id (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 place_type (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 1442, 32)             1068640   ['place_review[0][0]']        
                                                                                            

In [18]:
model.fit(x=[place_ids_encoded, place_types_encoded, pad_sequences(sequences, maxlen=max_review_length)], epochs=100, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=5)])

Epoch 1/100


ValueError: in user code:

    File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras/src/engine/training.py", line 1152, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras/src/engine/training.py", line 1106, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing.